<a href="https://colab.research.google.com/github/ramonsierrarey/TFM/blob/main/Dise%C3%B1asales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ventas de las tiendas vs distancia

### Código todo en uno ejecutable

In [2]:
import pandas as pd
import math

# Importamos las ventas por CP de nuestras tiendas (incluye nombre de la tienda, CP de la tienda, CP de la venta, cantidad vendida):
sales = pd.read_csv('Sales_per_point.csv',sep=",",decimal=".")

# Importamos las coordenadas de todos los CP de España:
coord = pd.read_csv('Codigos_Postales.csv',sep=",",decimal=".", dtype={'CP':'int', 'Market Size':'float','Latitude':'float','Longitude':'float'})

# Incluimos para cada CP de la tienda y CP de venta sus coordenadas:
sales2 = sales.merge(coord, left_on='CP CMP', right_on='CP', how='left')
sales2.rename(columns={'Latitude':'lata', 'Longitude':'lona'},inplace=True)
sales2.drop(['CP','Market Size'], inplace=True, axis=1)

df = sales2.merge(coord, left_on='CP Sales', right_on='CP', how='left')
df.rename(columns={'Latitude':'latb', 'Longitude':'lonb'},inplace=True)
df.drop('CP', inplace=True, axis=1)

# Calculamos la distancia entre el CP de la tienda y de la venta basándonos en las coordenadas:
df2 = pd.DataFrame(columns=['CMP', 'CP1', 'CP2', 'Dist', 'Roundist', 'Sales', 'Market Size'])
line=1

for cp,row in df.iterrows():
    cmp = row[1]
    cp1 = row[2]
    cp2 = row[3]
    sales = row[4]
    ms = row[7]
    lata = row[5]
    lona = row[6]
    latb = row[8]
    lonb = row[9]
    try:
        dist = math.acos(math.cos(math.radians(90-lata)) * math.cos(math.radians(90-latb)) + math.sin(math.radians(90-lata)) * math.sin(math.radians(90-latb)) * math.cos(math.radians(lona-lonb))) * 6371
    except:
        dist = 0
    rdist = round(dist)
    df2.loc[line] = (cmp, cp1, cp2, dist, rdist, sales, ms)
    line +=1 
    
# Agrupamos las ventas por distancia
Dist_sales = df2.groupby(['CMP','Roundist'],as_index=False).agg({'Sales':'sum', 'Market Size':'sum'}) 

#Añadimos las ventas acumuladas y el % que supone del total. Eliminamos aquellos CP demasiado alejados
Dist_sales['Pct'] = (Dist_sales['Sales']/Dist_sales['Sales'].sum())
Dist_sales['Acc_sales'] = Dist_sales['Sales'].cumsum()
Dist_sales['Acc_Pct'] = (Dist_sales['Acc_sales']/Dist_sales['Sales'].sum())
Dist_sales['Acc_MSize'] = Dist_sales['Market Size'].cumsum()
Dist_sales['MShare'] = Dist_sales['Sales']/Dist_sales['Market Size']
Dist_sales['Acc_MShare'] = Dist_sales['Acc_sales']/Dist_sales['Acc_MSize']

CMP_Dist_Sales = Dist_sales[Dist_sales.Roundist < 21]

# Exportamos el resultado a un CSV
CMP_Dist_Sales.to_csv('CMP_Dist_sales.csv', sep=',',decimal=".", index=False)

### Ejercicio por partes

Juntamos las ventas por CP por cada Punto de venta y las coordenadas, para saber la venta por distancia y el tamaño del mercado

In [ ]:
import pandas as pd
import math

# Importamos las ventas por CP de nuestras tiendas (incluye nombre de la tienda, CP de la tienda, CP de la venta, cantidad vendida):
sales = pd.read_csv('Sales_per_point.csv',sep=",",decimal=".")

# Importamos las coordenadas de todos los CP de España:
coord = pd.read_csv('Codigos_Postales.csv',sep=",",decimal=".", dtype={'CP':'int', 'Market Size':'float','Latitude':'float','Longitude':'float'})

# Incluimos para cada CP de la tienda y CP de venta sus coordenadas:
sales2 = sales.merge(coord, left_on='CP CMP', right_on='CP', how='left')
sales2.rename(columns={'Latitude':'lata', 'Longitude':'lona'},inplace=True)
sales2.drop(['CP','Market Size'], inplace=True, axis=1)

df = sales2.merge(coord, left_on='CP Sales', right_on='CP', how='left')
df.rename(columns={'Latitude':'latb', 'Longitude':'lonb'},inplace=True)
df.drop('CP', inplace=True, axis=1)

# Calculamos la distancia entre el CP de la tienda y de la venta basándonos en las coordenadas:
df2 = pd.DataFrame(columns=['CMP', 'CP1', 'CP2', 'Dist', 'Roundist', 'Sales', 'Market Size'])
line=1

for cp,row in df.iterrows():
    cmp = row[1]
    cp1 = row[2]
    cp2 = row[3]
    sales = row[4]
    ms = row[7]
    lata = row[5]
    lona = row[6]
    latb = row[8]
    lonb = row[9]
    try:
        dist = math.acos(math.cos(math.radians(90-lata)) * math.cos(math.radians(90-latb)) + math.sin(math.radians(90-lata)) * math.sin(math.radians(90-latb)) * math.cos(math.radians(lona-lonb))) * 6371
    except:
        dist = 0
    rdist = round(dist)
    df2.loc[line] = (cmp, cp1, cp2, dist, rdist, sales, ms)
    line +=1 
df2.head()

,CMP,CP1,CP2,Dist,Roundist,Sales,Market Size
1,Centro 1,3015,2001,137.564463,138,432.1,2.820383e+06
2,Centro 1,3015,2005,138.722344,139,134.4,6.728934e+06
3,Centro 1,3015,2006,149.034177,149,1035.6,4.895404e+06
4,Centro 1,3015,2008,134.155501,134,50.8,2.921140e+06
5,Centro 1,3015,2480,160.273548,160,3127.4,1.761668e+05


Comprobaciones. Distancia a la que vendemos >90%, y el MSize caiga por debajo del 5%

In [ ]:
# Agrupamos las ventas por distancia
Dist_sales = df2.groupby(['Roundist'],as_index=False).agg({'Sales':'sum', 'Market Size':'sum'}) 

#Añadimos las ventas acumuladas y el % que supone del total
Dist_sales['Pct'] = (Dist_sales['Sales']/Dist_sales['Sales'].sum())
Dist_sales['Acc_sales'] = Dist_sales['Sales'].cumsum()
Dist_sales['Acc_Pct'] = (Dist_sales['Acc_sales']/Dist_sales['Sales'].sum())
Dist_sales['Acc_MSize'] = Dist_sales['Market Size'].cumsum()
Dist_sales['MShare'] = Dist_sales['Sales']/Dist_sales['Market Size']
Dist_sales['Acc_MShare'] = Dist_sales['Acc_sales']/Dist_sales['Acc_MSize']

Dist_sales.head(40)

,Roundist,Sales,Market Size,Pct,Acc_sales,Acc_Pct,Acc_MSize,MShare,Acc_MShare
0,0,18247709.90,9.485188e+07,0.105233,1.824771e+07,0.105233,9.485188e+07,0.192381,0.192381
1,1,10367043.70,6.570463e+07,0.059786,2.861475e+07,0.165018,1.605565e+08,0.157783,0.178222
2,2,18915062.10,1.255016e+08,0.109081,4.752982e+07,0.274099,2.860581e+08,0.150716,0.166154
3,3,19376566.00,1.605747e+08,0.111743,6.690638e+07,0.385842,4.466328e+08,0.120670,0.149802
4,4,13494716.62,1.448409e+08,0.077823,8.040110e+07,0.463665,5.914737e+08,0.093169,0.135934
5,5,14753732.20,1.226094e+08,0.085083,9.515483e+07,0.548748,7.140832e+08,0.120331,0.133255
6,6,10213810.40,9.814267e+07,0.058902,1.053686e+08,0.607650,8.122258e+08,0.104071,0.129728
7,7,10348154.04,9.151845e+07,0.059677,1.157168e+08,0.667327,9.037443e+08,0.113072,0.128042
8,8,5237969.00,5.263608e+07,0.030207,1.209548e+08,0.697533,9.563804e+08,0.099513,0.126471
9,9,5629054.66,7.094205e+07,0.032462,1.265838e+08,0.729996,1.027322e+09,0.079347,0.123217


Exportamos para el siguiente paso

In [ ]:
# Agrupamos las ventas por distancia
Dist_sales = df2.groupby(['CMP','Roundist'],as_index=False).agg({'Sales':'sum', 'Market Size':'sum'}) 

#Añadimos las ventas acumuladas y el % que supone del total
Dist_sales['Pct'] = (Dist_sales['Sales']/Dist_sales['Sales'].sum())
Dist_sales['Acc_sales'] = Dist_sales['Sales'].cumsum()
Dist_sales['Acc_Pct'] = (Dist_sales['Acc_sales']/Dist_sales['Sales'].sum())
Dist_sales['Acc_MSize'] = Dist_sales['Market Size'].cumsum()
Dist_sales['MShare'] = Dist_sales['Sales']/Dist_sales['Market Size']
Dist_sales['Acc_MShare'] = Dist_sales['Acc_sales']/Dist_sales['Acc_MSize']

CMP_Dist_Sales = Dist_sales[Dist_sales.Roundist < 21]
CMP_Dist_Sales.head(30)

,CMP,Roundist,Sales,Market Size,Pct,Acc_sales,Acc_Pct,Acc_MSize,MShare,Acc_MShare
0,Centro 1,0,824968.4,3.909696e+06,0.004758,824968.4,0.004758,3.909696e+06,0.211006,0.211006
1,Centro 1,2,2219772.3,1.614578e+07,0.012801,3044740.7,0.017559,2.005548e+07,0.137483,0.151816
2,Centro 1,3,1285216.1,1.054722e+07,0.007412,4329956.8,0.024970,3.060270e+07,0.121853,0.141489
3,Centro 1,4,1184980.4,9.363152e+06,0.006834,5514937.2,0.031804,3.996586e+07,0.126558,0.137991
4,Centro 1,5,3512893.1,1.689425e+07,0.020258,9027830.3,0.052063,5.686010e+07,0.207934,0.158773
5,Centro 1,6,1630604.0,1.240095e+07,0.009404,10658434.3,0.061466,6.926105e+07,0.131490,0.153888
6,Centro 1,7,588559.0,4.148842e+06,0.003394,11246993.3,0.064860,7.340989e+07,0.141861,0.153208
7,Centro 1,9,892361.2,4.365727e+06,0.005146,12139354.5,0.070006,7.777562e+07,0.204402,0.156082
8,Centro 1,10,158948.2,8.268881e+05,0.000917,12298302.7,0.070923,7.860251e+07,0.192225,0.156462
9,Centro 1,14,1708117.4,8.304489e+06,0.009851,14006420.1,0.080774,8.690700e+07,0.205686,0.161166


In [ ]:
# Exportamos el resultado a un CSV
CMP_Dist_Sales.to_csv('CMP_Dist_sales.csv', sep=',',decimal=".", index=False)

PermissionError: [Errno 13] Permission denied: 'CMP_Dist_sales.csv'